# Results from the HTRC Harvard Collection

We ran the two extraction runs - total counts for all tokens, and counts for tokens that appear on the same pages with "literature" - on the 820k Harvard volumes in the HTRC data.

~43.5 billion total observed tokens:

In [1]:
from hol.queries import CountQueries

cq = CountQueries()

print(cq.total_token_count())

43401065888


9547 unique tokens (out of a list the 10k most frequent tokens in large modern corpora):

In [ ]:
tokens = cq.tokens()
print(len(tokens))

9547


Total volume from 1750-1920:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

ts = cq.baseline_series(range(1750, 1920))

plt.title('Per-year counts, all words')
plt.xlabel('Year')
plt.ylabel('Count')
plt.plot(*zip(*ts))

Total counts for "literature" and "poetry":

In [ ]:
years = range(1750, 1920)

literature = cq.token_series('literature', years)
poetry = cq.token_series('poetry', years)

plt.title('Raw per-year counts')
plt.xlabel('Year')
plt.ylabel('Count')
plt.plot(*zip(*literature), label='literature')
plt.plot(*zip(*poetry), label='poetry')
plt.legend(loc='upper right')

Per-million counts for "literature" and "poetry":

In [ ]:
years = range(1750, 1920)

literature = cq.token_wpm_series('literature', years)
poetry = cq.token_wpm_series('poetry', years)

plt.title('Volume-scaled per-year counts')
plt.xlabel('Year')
plt.ylabel('Count')
plt.plot(*zip(*literature), label='literature')
plt.plot(*zip(*poetry), label='poetry')
plt.legend(loc='upper right')